In [1]:
# %matplotlib notebook
import sympy as sym
from sympy import Matrix
from sympy.matrices.dense import hessian
from sympy import pprint
import numpy as np
import scipy as sp

from scipy import optimize as opt

import matplotlib.pyplot as plt

import ipyvolume as ipv
gradient = lambda f, v: Matrix([f]).jacobian(v)

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')

x_init = [0,0]
# Constraint
f_ = (1-x1)**2 + 100*(x2-x1)**2
g_ = (x1+1)**2 + 2*x2*(x2-1)**2

v = list(sym.ordered(f.free_symbols))
# f_ = lambda x: f.subs(zip(v, x))
f_ = sym.lambdify(v, f)
g_ = sym.lambdify(v, g)
print("v:", v)
print(f)
print(f_(*x_init))

print("Gradient:")
# Gs = lambda x: gradient(f=f,v=v).subs(zip(v,x))
Df = sym.lambdify(v, Matrix([f]).jacobian(v))
Dg = sym.lambdify(v, Matrix([g]).jacobian(v))

# KKT = 
print("Gradient at:", x_init)
print(Gs(*x_init).squeeze())

# print("Hessian:")
# Hs = sym.lambdify(v, sym.simplify(hessian(f=f,varlist=v)))
print("Hessian at:", x_init)
print(Hs(*x_init))

NameError: name 'f' is not defined

In [ ]:
from sympy.solvers import solve
from sympy import Symbol 
x1 = Symbol('x1') 
x2 = Symbol('x2') 
solve(x**2 - 1, x)

solve(g, x1, x2)


In [ ]:
### Initial conditions
xk = x_init
gk = Gs(*xk).squeeze()

k = 1
done = False
maxiter = 10
# maxiter = 10

t_max = 2

In [ ]:
### Gradient method

def minimizer(xk, axs, fig, method='newt', maxiter=100):
    # update = "broyden"
    update = "bfgs"
    print('k', 'tk', 'xk', '||gk||')
    # print(f'| k{:2} | tk{:^5} | xk{:^5} {:^5} | ||gk||{:^5}')
    itr = 0
    
    # penalty = sym.functions.special.delta_functions.Heaviside(g)*g**2 # Barrier function
    penalty = g**2 # Barrier function
    penalty_ = sym.lambdify(v, penalty)
    pen_done = True
    r = 1
    while pen_done:
        p = f + r*penalty
        # p = f
        fs = sym.lambdify(v, p)
        while itr < maxiter:
            itr += 1
            
            gk = Gs(*xk).squeeze()
            gknorm = np.linalg.norm(gk)
            
            if method == 'newt':
                H = Hs(*xk) # Hessian
                dk = -sp.linalg.inv(H)@gk
                tk = 1
            elif method == 'grad':
                dk = -gk
                # tk = opt.fminbound(lambda t_, xk=xk, dk=dk: fs(x=xk+t_*dk), 0, t_max)
                tk = opt.fminbound(lambda t_, xk=xk, dk=dk: fs(*(xk+t_*dk)), 0, t_max)
            elif method =='mod_newt':
                H = Hs(*xk) # Hessian
                [D, T] = np.linalg.eig(H)
                # import ipdb; ipdb.set_trace()
                mu = -1.01*np.min([np.min(D), 0])
                G = H+mu*np.eye(2) # Modified hessian positive semi-definite.
                dk = -sp.linalg.inv(G)@gk
                # tk = opt.fminbound(lambda t_, xk=xk, dk=dk: fs(x=xk+t_*dk), 0, t_max)
                tk = opt.fminbound(lambda t_, xk=xk, dk=dk: fs(*(xk+t_*dk)), 0, t_max)
            elif method == 'quasi_newt':
                if update == "bfgs":
                    if itr == 1:
                        H = np.random.rand(2,2)
                        # H = np.eye(2,2)
                        gk_old = 0.1
                    else:
                        sk = (xk - xk_old)[...,None]
                        yk = (gk - gk_old)[...,None]
                        if yk.T@sk <= 0:
                            print("Positive definiteness not guaranteed, try another init H")
                            print("sk:",  sk)
                            print("yk.T:",  yk.T)
                            print("yk.T@sk:",  yk.T@sk)
                            print("xk_old:", xk_old)
                            print("xk1:", xk1)
                            print("Hessian:", H)
                            break
                        
                        H = H + (yk@yk.T)/(yk.T@sk) - (H@(sk@sk.T)@H)/(sk.T@H@sk)
                        gk_old = gk
                    dk = -sp.linalg.inv(H)@gk
                elif update == "broyden":
                    if itr == 1:
                        # Bk = np.eye(2)
                        # Bk = np.array([[10, 0],[0,.9]])
                        Bk = np.random.rand(2,2)*10
                        dk = Bk@gk
                        gk_old = gk
                    else:
                        sk = (xk1 - xk)[...,None]
                        yk = (gk - gk_old)[...,None]
                        Bk1 = Bk + ((sk-Bk@yk)@(sk-Bk@yk).T)/((sk-Bk@yk).T@yk)
                            
                        dk = Bk1@gk
                elif update == "secant":
                    
                    ### TODO
                    if itr == 1:
                        # H = np.random.rand(2,2)
                        H = np.eye(2,2)
                        gk_old = 0
                        xk1 = xk
                        xk = 0
                    
                tk = opt.fminbound(lambda t_, xk=xk, dk=dk: fs(x=xk+t_*dk), 0, t_max)
                
            xk1 = xk+tk*dk
            xk_old = xk
            
            data = [itr, tk, *xk, gknorm]
            
            if itr < 20 or itr % 50 == 0:
                print('| {:2} | {:.5f} | {:.5f} {:.5f} | {:.5f} '.format(*data))
            
            axs.plot(xk[0], xk[1], xk1[0], xk1[1], marker = 'o', color='red', markersize=2)
            axs.plot([xk[0], xk1[0]], [xk[1], xk1[1]], color='red')
            fig.canvas.draw()
            
            xk = xk1
            
            if gknorm < 1e-6:
                print("Converged")
                return xk
        
         
        if penalty_(*xk) < 1e-6:
            print("Minimum penalty reached")
            pen_done = False
        else:
            r = r*10
         
    return xk
            
            

### Penalty stuff
penalty = g**2 # Barrier function
r = 1
p = f + r*penalty
fs = sym.lambdify(v, p)
        
# %matplotlib qt
# import matplotlib as mpl
# mpl.interactive(True)
fig_cont, axs_cont = plt.subplots(1)
w = 8
h = 8
step = 0.05
[X, Y] = np.meshgrid(np.arange(start=-w//2,step=step, stop=w//2), np.arange(start=-w//2,step=step,stop=w//2));
Z = np.array([fs(*(x, y)) for (x, y) in zip(X,Y)]);


# method = 'newt'
# method = 'mod_newt'
# method = 'quasi_newt'
method = 'grad'
x_init = [-1, 1]
x = minimizer(xk=x_init, axs=axs_cont, fig=fig_cont, method=method, maxiter=maxiter)
print("Final f(xk):", fs(x))

import time
time.sleep(1)
axs_cont.contour(X, Y, Z, levels=100)


In [ ]:
# https://github.com/widgetti/ipyvolume
# Gives error
ipv.figure()
ipv.plot_surface(X, Z, Y, color="orange")

In [ ]:
import numpy as np
from numpy.random import default_rng
import cv2

import matplotlib.pyplot as plt

SEED = 431313
rng = default_rng(SEED)

P = rng.random(100); print(P.shape); print(P.max(), P.min())
P = np.sort(P)

HSV = 0.5*np.ones((P.shape[0], 3)); print(HSV.shape)
# HSV = rng.random((P.shape[0], 3)); print(HSV.shape)

HSV[:,0] = P*np.pi; print(HSV[:10])

RGB = cv2.cvtColor(HSV.reshape(10,10,3).astype(np.float32), cv2.COLOR_HSV2RGB).reshape(100,-1)
print(RGB[:10])

fig, axs = plt.subplots(1,2)
axs[0].imshow(HSV.reshape(10,10,3), cmap='hsv')
axs[1].imshow(RGB.reshape(10,10,3))

print("FIRST")
print(HSV[:10])
print("LAST")
print(HSV[-10:])